In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import pickle
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from datetime import datetime

In [2]:
df = pd.read_csv("fine_food_dataset.csv")

In [3]:
df.shape

(40000, 7)

In [4]:
df.columns

Index(['Unnamed: 0', 'Text', 'Score', 'sentiment',
       'review_after_removing_html_tags', 'review_after_removing_punctuation',
       'review_after_pos_and_lemmatization'],
      dtype='object')

In [5]:
df.isna().sum()

Unnamed: 0                            0
Text                                  0
Score                                 0
sentiment                             0
review_after_removing_html_tags       0
review_after_removing_punctuation     0
review_after_pos_and_lemmatization    0
dtype: int64

In [6]:
df[ df.sentiment == 1 ].shape

(20000, 7)

In [7]:
df[ df.sentiment == 0 ].shape

(20000, 7)

In [9]:
tfidf_model = TfidfVectorizer()

X = tfidf_model.fit_transform(df["review_after_pos_and_lemmatization"])
y = df["sentiment"]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
y_train[ y_train == 1 ].shape

(16028,)

In [12]:
y_train[ y_train == 0 ].shape

(15972,)

In [13]:
y_test[ y_test == 1 ].shape

(3972,)

In [14]:
y_test[ y_test == 0 ].shape

(4028,)

#### KFold cross-validation

In [15]:
# Set up k-fold cross-validation
kf = KFold(n_splits=4, shuffle=True)

random_forest_scores = []
logistic_scores = []
svm_scores = []
gd_boost_scores = []

# Generate indices for each fold
for train_index, test_index in kf.split(X):
    x_train_data, x_test_data = X[train_index], X[test_index]
    y_train_data, y_test_data = y[train_index], y[test_index]

    ## Random Forest
    model_forest = RandomForestClassifier(max_depth=10, n_estimators=500)

    print("Fitting random forest to training data....")    
    model_forest = model_forest.fit(x_train_data, y_train_data)
    
    y_pred_data = model_forest.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    random_forest_scores.append(score)
    print(random_forest_scores)
    
    ## Logistic Regression
    
    model_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)
    print("Fitting Logistic Regression to training data....")  
    model_logistic_regression = model_logistic_regression.fit(x_train_data, y_train_data)
    
    y_pred_data = model_logistic_regression.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    logistic_scores.append(score)
    print(logistic_scores)
    
    ## SVM
    model_svm = SVC(C=0.1, kernel='linear')
    print("Fitting SVM to training data....")  
    model_svm = model_svm.fit(x_train_data, y_train_data)
    
    y_pred_data = model_svm.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    svm_scores.append(score)
    print(svm_scores)
    
    ## Gradient Boosting
    gb_classifier = GradientBoostingClassifier(random_state=2)
    print("Fitting Gradient Boosting to training data....")    
    gb_classifier.fit(x_train_data, y_train_data)
    
    y_pred_data = gb_classifier.predict(x_test_data)
    score = accuracy_score(y_test_data, y_pred_data)
    gd_boost_scores.append(score)
    print(gd_boost_scores)
    
    print("=======================================================")

# print(f"Cross-validation scores: {scores}")
# print(f"Average score: {np.mean(scores)}")

Fitting random forest to training data....
[0.8015]
Fitting Logistic Regression to training data....
[0.7941]
Fitting SVM to training data....
[0.86]
Fitting Gradient Boosting to training data....
[0.7878]
Fitting random forest to training data....
[0.8015, 0.794]
Fitting Logistic Regression to training data....
[0.7941, 0.8036]
Fitting SVM to training data....
[0.86, 0.8638]
Fitting Gradient Boosting to training data....
[0.7878, 0.7909]
Fitting random forest to training data....
[0.8015, 0.794, 0.8084]
Fitting Logistic Regression to training data....
[0.7941, 0.8036, 0.7962]
Fitting SVM to training data....
[0.86, 0.8638, 0.8569]
Fitting Gradient Boosting to training data....
[0.7878, 0.7909, 0.7833]
Fitting random forest to training data....
[0.8015, 0.794, 0.8084, 0.8055]
Fitting Logistic Regression to training data....
[0.7941, 0.8036, 0.7962, 0.7939]
Fitting SVM to training data....
[0.86, 0.8638, 0.8569, 0.8602]
Fitting Gradient Boosting to training data....
[0.7878, 0.7909, 0.7

In [16]:
print("Random Forest KFold cross-validation")
print(f"Cross-validation scores: {random_forest_scores}")
print(f"Average score: {np.mean(random_forest_scores)}\n")
print("Logistic Regression KFold cross-validation")
print(f"Cross-validation scores: {logistic_scores}")
print(f"Average score: {np.mean(logistic_scores)}\n")
print("SVM cross-validation")
print(f"Cross-validation scores: {svm_scores}")
print(f"Average score: {np.mean(svm_scores)}\n")
print("Gradient Boosting KFold cross-validation")
print(f"Cross-validation scores: {gd_boost_scores}")
print(f"Average score: {np.mean(gd_boost_scores)}\n")

Random Forest KFold cross-validation
Cross-validation scores: [0.8015, 0.794, 0.8084, 0.8055]
Average score: 0.80235

Logistic Regression KFold cross-validation
Cross-validation scores: [0.7941, 0.8036, 0.7962, 0.7939]
Average score: 0.79695

SVM cross-validation
Cross-validation scores: [0.86, 0.8638, 0.8569, 0.8602]
Average score: 0.860225

Gradient Boosting KFold cross-validation
Cross-validation scores: [0.7878, 0.7909, 0.7833, 0.7838]
Average score: 0.7864500000000001



#### Random Forest

In [17]:
# Start the timer
start_time = datetime.now()

model_forest = RandomForestClassifier(max_depth=10, n_estimators=500)

print("Fitting random forest to training data....")    
model_forest = model_forest.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Random Forest Training Runtime:', runtime)

Fitting random forest to training data....
Random Forest Training Runtime: 0:00:35.340942


In [18]:
model_forest.score(X_test, y_test)

0.799125

In [19]:
model_forest.score(X_train, y_train)

0.839125

In [20]:
# Start the timer
start_time = datetime.now()

y_pred = model_forest.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Random Forest Prediction Runtime:', runtime)

Random Forest Prediction Runtime: 0:00:00.690176


In [21]:
y_pred = cross_val_predict(model_forest, X_test, y_test)

In [22]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.83      0.82      4028
           1       0.82      0.79      0.81      3972

    accuracy                           0.81      8000
   macro avg       0.81      0.81      0.81      8000
weighted avg       0.81      0.81      0.81      8000



In [23]:
print(confusion_matrix(y_test, y_pred))

[[3355  673]
 [ 815 3157]]


In [24]:
filename = 'models/rd_tfidf_ml_model.sav'
pickle.dump(model_forest, open(filename, 'wb'))

#### Logistic Regression

In [25]:
# Start the timer
start_time = datetime.now()

model_logistic_regression = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)

print("Fitting Logistic Regression to training data....")  
model_logistic_regression = model_logistic_regression.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Logistic Regression Training Runtime:', runtime)

Fitting Logistic Regression to training data....
Logistic Regression Training Runtime: 0:00:00.235770


In [26]:
model_logistic_regression.score(X_test, y_test)

0.79775

In [27]:
model_logistic_regression.score(X_train, y_train)

0.80365625

In [28]:
# Start the timer
start_time = datetime.now()

y_pred = model_logistic_regression.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Logistic Regression Prediction Runtime:', runtime)

Logistic Regression Prediction Runtime: 0:00:00


In [29]:
y_pred = cross_val_predict(model_logistic_regression, X_test, y_test)

In [30]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      0.71      0.69      4028
           1       0.69      0.65      0.67      3972

    accuracy                           0.68      8000
   macro avg       0.68      0.68      0.68      8000
weighted avg       0.68      0.68      0.68      8000



In [31]:
print(confusion_matrix(y_test, y_pred))

[[2866 1162]
 [1396 2576]]


In [32]:
filename = 'models/lg_tfidf_ml_model.sav'
pickle.dump(model_logistic_regression, open(filename, 'wb'))

#### SVM

In [33]:
# Start the timer
start_time = datetime.now()

model_svm = SVC(C=0.1, kernel='linear')

model_svm = model_svm.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:04:06.061968


In [34]:
model_svm.score(X_test, y_test)

0.862125

In [35]:
model_svm.score(X_train, y_train)

0.87840625

In [36]:
# Start the timer
start_time = datetime.now()

y_pred = model_svm.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:00:53.653267


In [37]:
y_pred = cross_val_predict(model_svm, X_test, y_test)

In [38]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.86      0.84      4028
           1       0.85      0.80      0.83      3972

    accuracy                           0.83      8000
   macro avg       0.83      0.83      0.83      8000
weighted avg       0.83      0.83      0.83      8000



In [39]:
print(confusion_matrix(y_test, y_pred))

[[3462  566]
 [ 775 3197]]


In [40]:
filename = 'models/svm_tfidf_ml_model.sav'
pickle.dump(model_svm, open(filename, 'wb'))

#### Gradient Boosting Classifier

In [41]:
# Start the timer
start_time = datetime.now()


# create a Gradient Boosting classifier with default hyperparameters
gb_classifier = GradientBoostingClassifier(random_state=2)

# fit the classifier to the training data
gb_classifier.fit(X_train, y_train)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('Gradient Boosting Prediction Runtime:', runtime)


Gradient Boosting Prediction Runtime: 0:03:24.646710


In [42]:
gb_classifier.score(X_test, y_test)

0.782125

In [43]:
gb_classifier.score(X_train, y_train)

0.80153125

In [44]:
# Start the timer
start_time = datetime.now()

y_pred = gb_classifier.predict(X_test)

# Stop the timer
end_time = datetime.now()

# Calculate the runtime
runtime = end_time - start_time

# Print the runtime
print('SVM Prediction Runtime:', runtime)

SVM Prediction Runtime: 0:00:00.014249


In [45]:
y_pred = cross_val_predict(gb_classifier, X_test, y_test)

In [46]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.81      0.79      4028
           1       0.80      0.75      0.77      3972

    accuracy                           0.78      8000
   macro avg       0.78      0.78      0.78      8000
weighted avg       0.78      0.78      0.78      8000



In [47]:
filename = 'models/gb_boosting_tfidf_ml_model.sav'
pickle.dump(gb_classifier, open(filename, 'wb'))